# Image Embedding

In [3]:
from tensorflow.keras.applications.vgg16 import VGG16

Using TensorFlow backend.


In [4]:
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.vgg16 import preprocess_input
from tensorflow.keras.models import Model
import numpy as np

In [5]:
model = VGG16(include_top=True, weights='imagenet', input_tensor=None, input_shape=None, pooling=None, classes=1000)

In [6]:
# img_path = "./test_image.png"
# img = image.load_img(img_path, target_size=(224, 224))
# x = image.img_to_array(img)
# x = np.expand_dims(x, axis=0)
# x = preprocess_input(x)

In [7]:
# features = model.predict(x)
# fc2_features_extractor_model = Model(input=model.input, output=model.get_layer('fc2').output)

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:2: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("fc...)`
  


In [8]:
# fc2_features = fc2_features_extractor_model.predict(x)
# fc2_features = fc2_features.reshape((4096,))

In [9]:
# print(fc2_features.shape)

(4096,)


In [18]:
def get_embedding(img_path, model=VGG16(include_top=True, weights='imagenet', input_tensor=None, input_shape=None, pooling=None, classes=1000)):
    """
    Args:
        - model: trained image model (VGG16)
        - img_path: path to image
        
    Return:
        - (4096,) vector embedding of image
    """
    
    img = image.load_img(img_path, target_size=(224, 224))
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)
    
    features = model.predict(x)
    fc2_features_extractor_model = Model(input=model.input, output=model.get_layer('fc2').output)
    
    fc2_features = fc2_features_extractor_model.predict(x)
    fc2_features = fc2_features.reshape((4096,))
    
    return fc2_features

In [19]:
# print(get_embedding("./test_image.png").shape)

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:17: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("fc...)`


(4096,)


# Cosine Similarity

In [29]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

def get_cos_sims(target_img_id, img_id_list):
    """
    Args:
        - target_img_loc: location of input image 
        - img_loc_list: list of locations of images to be compared with target img
        
    Returns:
        - ndarray containing cosine similarity between target image and each image in img_loc_list
    """
    
    X = get_embedding(target_img_loc).reshape((1, 4096))
    Y = np.array([get_embedding(img) for img in img_loc_list])
    
    return cosine_similarity(X, Y, dense_output=True)

In [30]:
# sims = get_cos_sims("data/abstract_images/test_image.png", ["./test_image.png", "./test_image.png", "./test_image.png"])
# print(sims)

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:17: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("fc...)`


[[1.        1.        1.0000002]]


In [1]:
def get_closest_image(target_img_id, img_id_list):
    """
    Args:
        - target_img_loc: location of input image 
        - img_loc_list: list of locations of images to be compared with target img
        
    Returns:
        - location of image closest to target image
    """
    target_img_loc = img_id_to_path(target_img_id)
    img_loc_list = [img_id_to_path(img_id) for imd_id in img_id_list]
    cos_sims = list(get_cos_sims(target_img_loc, img_loc_list)[0])
    i = cos_sims.index(max(cos_sims))
    
    return img_loc_list[i]

In [36]:
# print(get_closest_image("./test_image.png", ["./test_image.png", "./test_image.png", "./test_image.png"]))

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:17: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("fc...)`


./test_image.png
